# **Heterogeneous Medical Knowledge Graph & Real-Time Object Detection**
This notebook covers:
1. **Medical Knowledge Graph Construction using Neo4j**
2. **Named Entity Recognition (NER) & Entity Normalization**
3. **Drug Interaction Prediction using Graph Neural Networks (GNNs)**
4. **Real-Time Compensation for Atmospheric Turbulence**
5. **Moving Object Detection using Computer Vision**


In [ ]:
# Install required libraries
!pip install neo4j pandas scikit-learn spacy torch dgl opencv-python numpy matplotlib
!python -m spacy download en_core_web_sm

## **1️⃣ Medical Knowledge Graph Construction (Neo4j)**

In [ ]:
from neo4j import GraphDatabase

# Connect to Neo4j
URI = "bolt://localhost:7687"  # Change if running remotely
AUTH = ("neo4j", "password")  # Update credentials

class MedicalGraph:
    def __init__(self, uri, auth):
        self.driver = GraphDatabase.driver(uri, auth=auth)

    def close(self):
        self.driver.close()

    def create_nodes(self, tx, node_type, properties):
        query = f"CREATE (n:{node_type} {{ {', '.join([f'{key}: ${key}' for key in properties.keys()])} }})"
        tx.run(query, **properties)

    def create_relationship(self, tx, node1, rel, node2):
        query = (
            f"MATCH (a:{node1['type']}), (b:{node2['type']}) "
            f"WHERE a.id = $id1 AND b.id = $id2 "
            f"CREATE (a)-[:{rel}]->(b)"
        )
        tx.run(query, id1=node1['id'], id2=node2['id'])


## **2️⃣ Named Entity Recognition (NER) with SpaCy**

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

sample_text = "John was prescribed Metformin for his Diabetes but experienced nausea."
print(extract_entities(sample_text))


## **3️⃣ Graph Neural Network (GNN) for Drug Interaction Prediction**

In [ ]:
!pip uninstall torch torchdata dgl -y
!pip install torch torchvision torchaudio
!pip install dgl

In [ ]:
!pip install dgl torchdata

In [ ]:
!pip uninstall torchdata

In [ ]:
!pip install dgl torch torchvision torchaudio tensorflow networkx numpy scipy


In [ ]:
!pip install torch-geometric
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html

In [ ]:
import torch
import torch.nn.functional as F
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.data import HeteroData
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx

# Create a Heterogeneous Graph with Drugs & Diseases
data = HeteroData()

# Add Drug Nodes
data['Drug'].x = torch.randn(3, 10)  # 3 Drugs with 10 features each

# Add Disease Nodes
data['Disease'].x = torch.randn(2, 10)  # 2 Diseases with 10 features each

# Define Relationships (Edges)
data['Drug', 'interacts_with', 'Drug'].edge_index = torch.tensor([[0, 1], [1, 2]])  # Drug-Drug interactions
data['Drug', 'treats', 'Disease'].edge_index = torch.tensor([[0, 1], [2, 0]])  # Drug-Disease treatment

# Define GNN Model
class DrugGNN(torch.nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(DrugGNN, self).__init__()
        self.conv1 = GCNConv(in_feats, hidden_feats)
        self.conv2 = GCNConv(hidden_feats, out_feats)

    def forward(self, x, edge_index):
        h = F.relu(self.conv1(x, edge_index))
        h = self.conv2(h, edge_index)
        return h

# Initialize & Test Model
model = DrugGNN(10, 16, 2)
output = model(data['Drug'].x, data['Drug', 'interacts_with', 'Drug'].edge_index)
print("Output shape:", output.shape)

# Convert PyG Heterogeneous Graph to NetworkX for Visualization
G = to_networkx(data.to_homogeneous(), to_undirected=True)

# Assign Colors Based on Node Type
node_colors = []
for node in G.nodes():
    if node < 3:  # First 3 nodes are Drugs
        node_colors.append("lightblue")
    else:  # Last 2 nodes are Diseases
        node_colors.append("lightcoral")

# Plot the Graph
plt.figure(figsize=(6, 6))
nx.draw(G, with_labels=True, node_color=node_colors, edge_color="gray", node_size=600, font_size=12)
plt.title("Drug-Disease Interaction Graph")
plt.show()

## **4️⃣ Real-Time Compensation for Atmospheric Turbulence**

In [ ]:
import cv2
import numpy as np

def stabilize_frames(frames):
    stabilized = []
    for i in range(1, len(frames)):
        flow = cv2.calcOpticalFlowFarneback(cv2.cvtColor(frames[i-1], cv2.COLOR_BGR2GRAY),
                                            cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY),
                                            None, 0.5, 3, 15, 3, 5, 1.2, 0)
        h, w = flow.shape[:2]
        flow_map = np.column_stack((np.repeat(np.arange(h), w), np.tile(np.arange(w), h))) + flow.reshape(-1, 2)
        stabilized.append(cv2.remap(frames[i], flow_map[:, 0], flow_map[:, 1], interpolation=cv2.INTER_LINEAR))
    return stabilized

## **5️⃣ Moving Object Detection**

In [ ]:
!pip install opencv-python

In [ ]:
fgbg = cv2.createBackgroundSubtractorMOG2()

cap = cv2.VideoCapture("video.mp4")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    fgmask = fgbg.apply(frame)
    cv2.imshow("Moving Objects", fgmask)
    if cv2.waitKey(30) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2

fgbg = cv2.createBackgroundSubtractorMOG2()
cap = cv2.VideoCapture("video.mp4")

frame_count = 0  # Counter to save a frame

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    fgmask = fgbg.apply(frame)

    # Save the 50th frame for visualization
    if frame_count == 50:
        cv2.imwrite("moving_objects.png", fgmask)
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()